In [1]:
import classla
import ctypes
from datetime import datetime
import pickle
import string
import pandas as pd

In [2]:
ctypes.cdll.LoadLibrary('caffe2_nvrtc.dll')

<CDLL 'caffe2_nvrtc.dll', handle 7ffa9e850000 at 0x1dad2163520>

## Get lemmas from conllu format as an array

In [3]:
def get_lemmas(conll):
    res=[]
    for line in conll.splitlines():
        if not line.startswith('#'):
            words = line.strip('\n').split('\t')
            if(len(words)>3):  
                res.append(words[2])
    return res

## Insert negation marker NEG

In [4]:
def filter_negation(lemma_array, text, negation_tokens, characters):
    res=[]
    words=text.split()
    index=0
    
    for lemma in lemma_array:
        if lemma not in characters:
            orginial_word=words[index]
            index+=1

            if orginial_word in negation_tokens:
                res.append('NEG')

        res.append(lemma)
    
    return ' '.join(res)      
    

## Find lemma for every word and set NEG markers for negation words

In [5]:
def lemmatization_and_negation(tweets_dataframe, negation_words, nlp):
    
    negation_tokens=negation_words.split()
    characters=".,!?'"
    translator=str.maketrans(characters, ' '*len(characters)) 
    tweets_dataframe['FilteredText']=''
    for index,row in tweets_dataframe.iterrows():
        doc=nlp(row['Text'])
        lemma_array=get_lemmas(doc.conll_file.conll_as_string())
        text=row['Text'].translate(translator)
        row['FilteredText']=filter_negation(lemma_array,text,negation_tokens,characters)
    return tweets_dataframe


## loop through tweets in blocks of 200 tweets and performe lemmatization and negation on tweet text

In [6]:
negation_words='nema nemamo neću ni necete nemate nemoj nećavši nisam necemo nemaš nemojmo nisu nisi neces nemaju neće niste stop necu nemam nećete nismo necavsi nemojte nećemo nije nece nit niti nećeš ne nemas'

tweets=pd.read_pickle("PickleDataframes\\Processed_Tweets.pkl")
number_of_tweets=len(tweets.index)
step=200
index=0
pok=0
nlp = classla.Pipeline('sr', type='nonstandard')

while index < number_of_tweets:
    top=min(index+step,number_of_tweets)
    df=tweets.iloc[index:top]
    new_df=lemmatization_and_negation(df, negation_words,nlp)
    file_name_pkl=''.join(['LemmaNegation\\LemmaTweetsData',str(pok),'.pkl'])
    new_df.to_pickle(file_name_pkl)
    print(pok)
    index+=step
    pok+=1


Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\DarkoSarajkic\\classla_resources\\sr_nonstandard_models\\sr_nonstandard_tokenizer.pt', 'lang': 'sr', 'shorthand': 'sr_nonstandard', 'mode': 'predict', 'type': 'nonstandard'}
---
Loading: ner
With settings: 
{'model_path': 'C:\\Users\\DarkoSarajkic\\classla_resources\\sr_nonstandard_models\\sr_nonstandard_ner.pt', 'pretrain_path': 'C:\\Users\\DarkoSarajkic\\classla_resources\\sr_set_models\\sr_set.pretrain.pt', 'forward_charlm_path': None, 'backward_charlm_path': None, 'lang': 'sr', 'shorthand': 'sr_nonstandard', 'mode': 'predict', 'type': 'nonstandard'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\DarkoSarajkic\\classla_resources\\sr_nonstandard_models\\sr_nonstandard_tagger.pt', 'pretrain_path': 'C:\\Users\\DarkoSarajkic\\classla_resources\\sr_set_models\\sr_set.pretrain.pt', 'lang': 'sr', 'shorthand': 'sr_nonstandard', 'mode': 'predict', 'type': 'nonstandard'}
---
Loading: lemma
With setti